# Никита Сысоев
# Задание 2

### 2.1
Для решения задачи классификации я применил полносвязную нейронную сеть и модель работы с текстом **"Bag of words"**. 

Считываются тексты книг из папки для обучения. Эти тексты разбиваются на предложения, и для каждого предложения отмечается автор. Каждое предложение приводится в "нормальный" вид: все слова ставятся в нормальную форму, удаляются знаки препинания и числа.

Заполняется словарь частот слов из обучающих предложений.
Каждое предложение преврящается в массив, в котором для каждого слова из словаря указана частота его употребления в данном предложении.

Полученные массивы подаются на вход полносвязной нейронной сети с одним скрытым слоем. Сеть обучается.

In [1]:
import numpy as np
import os
import re
from pymorphy2 import MorphAnalyzer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Embedding
from keras import regularizers
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.model_selection import train_test_split

RND_SEED = 7

np.random.seed(RND_SEED)

Using TensorFlow backend.


In [2]:
def get_text_fb2(path):
    """Returns parsed fb2 book as string"""
    text_file = open(path, encoding='utf8')
    text = text_file.read()
    
    #Remove marks
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'[\n\t]', ' ', text)
    return text

In [3]:
def scan_fb2_books(path):
    """Returns tuples of book's name and text for each fb2 book in directory"""
    books = []
    for f in os.listdir(path):
        if (f.endswith('.fb2')):
            books.append((f,get_text_fb2(path + f)))
    return books

In [4]:
def get_sentences(text):
    """Returns all sentences in text"""
    sentences = re.split(r' *[\.\?!…][\'"\)\]]* *', text)
    sentences = filter(lambda s: len(s) > 12, sentences)
    return sentences

In [5]:
def normalize(text, cache={}, morphy_analyzer=MorphAnalyzer()):
    """Normalizes text.
    
    Puts each word in it's normal form, removes all numbers, punctuation marks and words shorter than 4.
    
    Args:
        text (str): Text to normalize
        cache (dict): Dictionary of already known normal forms (speeds up the calculation)
        morphy_analyzer (MorphAnalyzer): Analyzer used for putting a word in in't normal form
    """
    text_without_numbers = re.sub(r'[0-9]+', '', text)
    words = text_to_word_sequence(text_without_numbers)
    text_norm = ''
    for word in words:
        w_low = word.lower()
        if w_low not in cache:
            #Put new word normal form in cache
            cache[w_low] = morphy_analyzer.parse(w_low)[0].normal_form
        if (len(cache[w_low]) > 3):
            text_norm += " " + cache[w_low]

    return text_norm

In [6]:
hegel_path = "./Data/Hegel/"
gogol_path = "./Data/Gogol/"

he_books = scan_fb2_books(hegel_path)
go_books = scan_fb2_books(gogol_path)

In [7]:
#All sentences of Hehel and Gogol
he_sents = [sent for book in he_books for sent in get_sentences(book[1])]
go_sents = [sent for book in go_books for sent in get_sentences(book[1])]

In [8]:
sample_size = 20000 #Number of sentences for training and testing
#Equal number of sentences by Hegel and Gogol
he_sents_sample = he_sents[:sample_size // 2]
go_sents_sample = go_sents[:sample_size // 2]

In [9]:
cache = {} #Cache for words normal form
morph_an = MorphAnalyzer() 
#Normalized sentences of Hegel and Gogol
he_sents_norm = [normalize(sent, cache, morph_an) for sent in he_sents_sample]
go_sents_norm = [normalize(sent, cache, morph_an) for sent in go_sents_sample]

In [10]:
sentences = he_sents_norm + go_sents_norm
#List of tuples (H,G), where H and G are probabilities of a text to be Hegel's and Gogol's
authors = []  
for sent in he_sents_norm:
    authors.append((1,0))
for sent in go_sents_norm:
    authors.append((0,1))
sentences = np.array(sentences)
authors = np.array(authors)

In [11]:
train_sents, test_sents, y_train, y_test = train_test_split(sentences, authors, train_size=0.8, random_state=321)

In [12]:
#Number of words in tokenizer's vocabulary
vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_sents)

In [13]:
#Using tokenizer to calculate the frequency of words in sentences for each word in tokenizer's vocabulary
X_train = tokenizer.texts_to_matrix(train_sents, mode='freq')
X_test = tokenizer.texts_to_matrix(test_sents, mode='freq')

In [14]:
model = Sequential()
model.add(Dense(200, input_shape=(vocab_size,), 
                activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, batch_size=16, verbose=1, epochs=5, validation_data=(X_test, y_test))

Train on 16000 samples, validate on 4000 samples
Epoch 1/5
16000/16000 [==============================] - 15s 958us/step - loss: 0.2838 - acc: 0.9005 - val_loss: 0.2060 - val_acc: 0.9223
Epoch 2/5
16000/16000 [==============================] - 6s 376us/step - loss: 0.1671 - acc: 0.9353 - val_loss: 0.1940 - val_acc: 0.9287
Epoch 3/5
16000/16000 [==============================] - 6s 390us/step - loss: 0.1451 - acc: 0.9432 - val_loss: 0.1923 - val_acc: 0.9305
Epoch 4/5
16000/16000 [==============================] - 6s 373us/step - loss: 0.1328 - acc: 0.9476 - val_loss: 0.1942 - val_acc: 0.9310
Epoch 5/5
16000/16000 [==============================] - 8s 469us/step - loss: 0.1220 - acc: 0.9517 - val_loss: 0.1950 - val_acc: 0.9317


В качестве меры качества использовалась **accuracy**, на тестовой выборке **accuracy=0.93**. 

Интерпретировать это можно как то, что модель работает хорошо, однако точная цифра мало что дает, поскольку качество зависит от размера текста - чем больше текст, тем выше качество. Поэтому 0.93 на коротких предложениях - это хороший результат.

### 2.2
Для вывода предложений, похожих и на Гоголя, и на Гегеля, предскажем вероятность для обоих классов с помощью нашей модели и выведем 30 предложений с наибольшим произведением вероятностей (таким образом в топе оказываются предложения с большими вероятностями обоих классов)

In [16]:
#Getting word frequency array for all sentences
sentences_mtx = tokenizer.texts_to_matrix(sentences, mode='freq')
pred = model.predict(sentences_mtx)

In [17]:
#Original sentences (unnormalized)
sentences_orig = he_sents_sample + go_sents_sample

In [18]:
#Sorting sentences by multiplication of probabilities (moving (1,1) to the top and (0,0) to the bottom)
sorted_by_uncert = [s for s,_ in sorted(zip(sentences_orig, pred), key=lambda pair: pair[1][1] * pair[1][0], reverse=True)]

In [19]:
show_uncert = 30
for s in range(show_uncert):
    print(sorted_by_uncert[s])

«В Афинах, как рассказывают, все выбежали из мастерских, чтобы посмотреть на него, и, когда ему кто-то сказал, что все смотрят на него с удивлением, как на невиданного зверя, он ответил: «нет, как на настоящего человека»[73]
Этот вопрос все снова и снова повторяется, снова и снова прибавляется одно зернышко или вырывается один волос
Книгу, в которой ничего не написано, каждый может понять
Если, например, исчезает человек, то нет ни рук, ни ног помимо названия, как, например, мы называем рукой каменную руку, ибо отрезанная рука все равно, что каменная»
Нужно вообще сказать, что здесь начинает все более и более бросаться в глаза рядоположность
Ибо по какому другому соображению никто из них не сделал землю элементом, как это делает народное представление
Подробности излагаются источниками различно и согласно некоторым из них, ему, между прочим, ставилась в вину хвалебная песнь Гермию и надпись на посвященной последнему статуе
Соблазнил ли он хоть кого-нибудь – и в особенности тех, с котор

Видно, что сюда попало несколько неправильно разбитых предложений, а также предложения, в которых речь идет о довольно абстрактных вещах, упоминаемых обоими авторами (душа, любовь, грамота т.д.)

-------------------------
Проверим нашу модель на книгах Гоголя и Гегеля, не участвовавших в обучении и тестировании

In [20]:
def predict(text, tokenizer, model):
    """Predicts probability for text to be Hegel's or Gogol's
    
    Args:
        text (str): Text for classification
        tokenizer (Tokenizer): Tokenizer with filled vocabulary
        model (Sequential): Model used to make prediction
    """
    text_norm = normalize(text)
    freqs = tokenizer.texts_to_matrix((text_norm,), mode='freq')
    return model.predict(freqs)[0]

In [21]:
#Books not involved in training
test_he_path = "./Test/hegel/"
test_go_path = "./Test/Gogol/"

test_he_books = scan_fb2_books(test_he_path)
test_go_books = scan_fb2_books(test_go_path)

In [22]:
print("Hegel:")
for book in test_he_books:
    print(book[0])
    print(predict(book[1], tokenizer, model))

print("Gogol:")
for book in test_go_books:
    print(book[0])
    print(predict(book[1], tokenizer, model))

Hegel:
Лекции по истории философии. Книга третья.fb2
[0.9967961  0.00320395]
Философия истории.fb2
[0.9930033  0.00699667]
Философия права.fb2
[0.99589986 0.00410013]
Gogol:
Том 4. Ревизор.fb2
[0.0118834 0.9881166]
Том 5. Женитьба. Драматические отчерки.fb2
[0.01230689 0.98769313]
Том 6. Мертвые души. Том 1.fb2
[0.01414921 0.98585075]
Том 7. Мертвые души. Том 2.fb2
[0.02441487 0.97558516]


Как можно видеть, большие тексты модель классифицирует вполне уверенно